# SSB Template Notebook

This notebook is a template for producing single-sideband modulated pulses using the RFSoC 4x2.

In [28]:
# Import libraries
%load_ext autoreload
%autoreload 2

from pulse_lib.pulse_program import *
import pickle

from qick import *
import Pyro4
%pylab inline

# Pyro communicates with board
Pyro4.config.SERIALIZER = "pickle"
Pyro4.config.PICKLE_PROTOCOL_VERSION=4

# Static IP proxy
ns_host = "172.24.106.21"
ns_port = 8888
proxy_name = "myqick"

# QICK object
ns = Pyro4.locateNS(host=ns_host, port=ns_port)
soc = Pyro4.Proxy(ns.lookup(proxy_name))
soccfg = QickConfig(soc.get_cfg())
print(soccfg)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib

QICK configuration:

	Board: RFSoC4x2

	Software version: 0.2.267
	Firmware timestamp: Wed Sep  6 18:49:29 2023

	Global clocks (MHz): tProcessor 409.600, RF reference 491.520

	2 signal generator channels:
	0:	axis_signal_gen_v6 - envelope memory 65536 samples (6.667 us)
		fs=9830.400 MHz, fabric=614.400 MHz, 32-bit DDS, range=9830.400 MHz
		DAC tile 0, blk 0 is DAC_B
	1:	axis_signal_gen_v6 - envelope memory 65536 samples (6.667 us)
		fs=9830.400 MHz, fabric=614.400 MHz, 32-bit DDS, range=9830.400 MHz
		DAC tile 2, blk 0 is DAC_A

	2 readout channels:
	0:	axis_readout_v2 - configured by PYNQ
		fs=4423.680 MHz, decimated=552.960 MHz, 32-bit DDS, range=4423.680 MHz
		maxlen 16384 accumulated, 1024 decimated (1.852 us)
		triggered by output 7, pin 14, feedback

In [29]:
# Import dictionary of sequences from pickle file
# with open('pulse_sequences/Sequence_ch1_ch2_mw.pickle', 'rb') as handle:
#     imported_sequences = pickle.load(handle)

# Test pulse sequence
imported_sequences = {}
imported_sequences["MW"] = [(5000, 1.0, 0.5, 0)]
# imported_sequences["DIG"] = [(1000, 1)]

In [30]:
dac_delay = 0
dac_gain = 20000

# Assign imported pulse sequence to DAC_A and DAC_B, setting gain and delays
pulse_sequences = PulseProgram(imported_sequences,
                              ch_map = {"DAC_A":"MW", "DAC_B":"MW"},
                              delays = {"DAC_A":dac_delay, "DAC_B":dac_delay},
                              gains  = {"DAC_A":dac_gain, "DAC_B":dac_gain},
                              )

----- DAC A -----
ch_type: DAC
ch_index: 1
delay: 0
gain: 20000
lengths: [5.0]
times: [0.0]
amps: [1.0]
freqs: [500.0]
phases: [0.0]
num_pulses: 1
duration: 5.0
----- DAC B -----
ch_type: DAC
ch_index: 0
delay: 0
gain: 20000
lengths: [5.0]
times: [0.0]
amps: [1.0]
freqs: [500.0]
phases: [0.0]
num_pulses: 1
duration: 5.0
----- End time: 5.0 -----


In [31]:
# Import DAC and ssb calibrations
with open('pickles/calibrate_phase.pickle', 'rb') as handle:
    delta_phis = pickle.load(handle)
with open('pickles/ssb_params_10000.pickle', 'rb') as handle:
    ssb_params = pickle.load(handle)

In [32]:
# Program RFSoC with pulse sequence and confiure for an external trigger (PMOD1_0)
prog = QickProgram(soccfg)
pulse_sequences.generate_asm(prog, delta_phis, ssb_params, reps=1)
pulse_sequences.config_external_start(soc, prog)
print(prog)


// Program

        synci 200;
        regwi 0, $14, 0;
LOOP_I: regwi 1, $22, 218453333;                //freq = 218453333
        regwi 1, $23, 0;                        //phase = 0
        regwi 1, $25, 19600;                    //gain = 19600
        regwi 1, $26, 592896;                   //phrst| stdysel | mode | | outsel = 0b01001 | length = 3072 
        regwi 1, $27, 0;                        //t = 0
        set 1, 1, $22, $23, $0, $25, $26, $27;  //ch = 1, pulse @t = $27
        regwi 0, $22, 218453333;                //freq = 218453333
        regwi 0, $23, 434849978;                //phase = 1739399913
        bitwi 0, $23, $23 << 2;
        mathi 0, $23, $23 + 1;
        regwi 0, $25, 20000;                    //gain = 20000
        regwi 0, $26, 592896;                   //phrst| stdysel | mode | | outsel = 0b01001 | length = 3072 
        regwi 0, $27, 0;                        //t = 0
        set 0, 0, $22, $23, $0, $25, $26, $27;  //ch = 0, pulse @t = $27
        waiti

## External Trigger Pins

![External trigger pins](images/external_trigger_pins.jpg)

In [33]:
# Stop DACs
soc.reset_gens()

In [38]:
# Simple implementation where DACs continuously output a fixed frequency

# SSB parameters
freq = 500
phases = ssb_params[freq]["phases"]
gains = ssb_params[freq]["gains"]

# Program DAC registers
prog = QickProgram(soccfg)
soc.reset_gens()
for ch in range(2): # Both DACs
    prog.declare_gen(ch=ch, nqz=1) # Initialise DAC
    prog.set_pulse_registers(ch = ch,
                             gain = gains[ch], # SSB gains
                             freq = prog.freq2reg(freq, gen_ch=ch),
                             phase = prog.deg2reg((phases[ch] # SSB frequency
                                                   + interpolate_phase(freq, delta_phis)[ch]), # Calibration frequency
                                                  gen_ch=ch),
                             style = "const",
                             length = prog.us2cycles(100, gen_ch=ch),
                             mode = "periodic", # Constant output
                            )
    prog.pulse(ch=ch, t=0) # Trigger DAC

# Start DACs
prog.config_all(soc) # Configure for internal start
soc.tproc.start() # Internal start
print(prog)


// Program

  regwi 0, $22, 218453333;                      //freq = 218453333
  regwi 0, $23, 434849978;                      //phase = 1739399913
  bitwi 0, $23, $23 << 2;
  mathi 0, $23, $23 + 1;
  regwi 0, $25, 10000;                          //gain = 10000
  regwi 0, $26, 913408;                         //phrst| stdysel | mode | | outsel = 0b01101 | length = 61440 
  regwi 0, $27, 0;                              //t = 0
  set 0, 0, $22, $23, $0, $25, $26, $27;        //ch = 0, pulse @t = $27
  regwi 1, $22, 218453333;                      //freq = 218453333
  regwi 1, $23, 0;                              //phase = 0
  regwi 1, $25, 9800;                           //gain = 9800
  regwi 1, $26, 913408;                         //phrst| stdysel | mode | | outsel = 0b01101 | length = 61440 
  regwi 1, $27, 0;                              //t = 0
  set 1, 1, $22, $23, $0, $25, $26, $27;        //ch = 1, pulse @t = $27


In [35]:
# Stop DACs
soc.reset_gens()